In [1]:
from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1660980033939_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Exploratory Analysis on Patent Data

In [2]:
patent_path = "s3://patents-analytics/raw/patent.tsv"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
patent = spark.read.options(header='True', delimiter='\t', inferSchema='True', escape="\"", multiline='True').csv(patent_path).repartition(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
patent.rdd.getNumPartitions()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10

In [5]:
patent.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(id='9074694', type='utility', number='9074694', country='US', date=datetime.datetime(2015, 7, 7, 0, 0), abstract='A control device for hydraulic media with a control valve, in particular, a proportional valve that has a sleeve housing provided with radial openings, wherein at least one pressurized flow feed line, at least one control line, and at least one return line are connected to the openings, and the opening for the pressurized flow feed line is connected to a ring-shaped groove, with a control slide that has control edges and is supported in the sleeve housing so that it can move in the axial direction and is actuated by an electromagnetically constructed control unit, advantageously for controlling a phase adjustment device for the variable adjustment of the control times of gas exchange valves of a reciprocating piston internal combustion engine. A non-return valve is installed in the pressurized flow feed line or in the ring-shaped groove that is arranged in the sleeve h

In [6]:
patent.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- number: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- abstract: string (nullable = true)
 |-- title: string (nullable = true)
 |-- kind: string (nullable = true)
 |-- num_claims: integer (nullable = true)
 |-- filename: string (nullable = true)
 |-- withdrawn: integer (nullable = true)

In [7]:
patent.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7991058

In [8]:
patent.createOrReplaceTempView('patent')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Analyse the field id

In [32]:
spark.sql("""
    SELECT COUNT(DISTINCT(id)) FROM patent
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+
|count(DISTINCT id)|
+------------------+
|           7991058|
+------------------+

### Analyse the field `type`

In [22]:
spark.sql("""
    SELECT type, count(*) FROM patent GROUP BY type
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------+
|                type|count(1)|
+--------------------+--------+
|                TVPP|       3|
|              design|  696387|
|             reissue|   20012|
|defensive publica...|     509|
|             utility| 7242132|
|statutory inventi...|    2259|
|               plant|   29756|
+--------------------+--------+

**Observations**:
- There is no missing data in patent type

### Analyse the field `number`

In [23]:
spark.sql("""
    SELECT COUNT(DISTINCT number) as unique_number FROM patent
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+
|unique_number|
+-------------+
|      7991058|
+-------------+

**Observations**:
- All the patent numbers are unique, no duplicates

### Analyze the field `country`

In [24]:
spark.sql("""
    SELECT DISTINCT(country) FROM patent
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+
|country|
+-------+
|     US|
+-------+

**Observations**:
    - All the patents are issued in US

### Analyze the field `date`

In [12]:
spark.sql("""
    SELECT YEAR(CAST(date as date)) as year, count(*)  FROM patent GROUP BY year ORDER BY year DESC LIMIT ALL
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------+
|year|count(1)|
+----+--------+
|2021|  363829|
|2020|  390572|
|2019|  392618|
|2018|  341104|
|2017|  352587|
|2016|  334674|
|2015|  326969|
|2014|  327014|
|2013|  303658|
|2012|  277285|
|2011|  248101|
|2010|  244599|
|2009|  192052|
|2008|  185260|
|2007|  182978|
|2006|  196489|
|2005|  157829|
|2004|  181413|
|2003|  187104|
|2002|  184494|
+----+--------+
only showing top 20 rows

### Analyse field abstract

In [13]:
spark.sql("""
    SELECT *  FROM patent WHERE abstract IS NULL OR abstract=''
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+-------+-------+-------------------+--------+--------------------+----+----------+-------------+---------+
|     id|  type| number|country|               date|abstract|               title|kind|num_claims|     filename|withdrawn|
+-------+------+-------+-------+-------------------+--------+--------------------+----+----------+-------------+---------+
|D768887|design|D768887|     US|2016-10-11 00:00:00|    null|          LED module|  S1|         1|ipg161011.xml|        0|
|D538744|design|D538744|     US|2007-03-20 00:00:00|    null|Electrical connector|  S1|         1|ipg070320.xml|        0|
|D697662|design|D697662|     US|2014-01-14 00:00:00|    null|       Cigar pendant|  S1|         1|ipg140114.xml|        0|
|D680819|design|D680819|     US|2013-04-30 00:00:00|    null|Cutter for making...|  S1|         1|ipg130430.xml|        0|
|D859388|design|D859388|     US|2019-09-10 00:00:00|    null|Case for mobile p...|  S1|         1|ipg190910.xml|        0|
|D547004|design|

In [15]:
spark.sql("""
    SELECT kind, count(*)  FROM patent WHERE abstract IS NULL OR abstract='' GROUP BY kind
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------+
|kind|count(1)|
+----+--------+
|  E1|     305|
|  S1|  439083|
|  B2|       4|
|  B1|       6|
|  H1|       6|
|   A|       3|
+----+--------+

**Observations**:
- There are several patents with null abstracts, mostly for S1 patent (Design Patent)

### Analyse field title

In [14]:
spark.sql("""
    SELECT *  FROM patent WHERE title IS NULL OR title=''
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+----+------+-------+----+--------+-----+----+----------+--------+---------+
| id|type|number|country|date|abstract|title|kind|num_claims|filename|withdrawn|
+---+----+------+-------+----+--------+-----+----+----------+--------+---------+
+---+----+------+-------+----+--------+-----+----+----------+--------+---------+

**Observations**:
- All the patents have a title

### Analyse field kind

In [17]:
spark.sql("""
    SELECT kind, count(*)  FROM patent GROUP BY kind
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------+
|kind|count(1)|
+----+--------+
|  E1|   11320|
|  S1|  486805|
|  I5|       3|
|  B2| 4162294|
|   S|  209582|
|  I4|     509|
|  B1|  688460|
|   P|    8282|
|   H|    1982|
|  H2|       3|
|  H1|     273|
|  P2|   14787|
|   E|    8692|
|  P3|    6687|
|   A| 2391379|
+----+--------+

### Analyse field num_claims

In [18]:
spark.sql("""
    SELECT min(num_claims), max(num_claims)  FROM patent 
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+---------------+
|min(num_claims)|max(num_claims)|
+---------------+---------------+
|              1|            887|
+---------------+---------------+

### Analyse field filename

In [19]:
spark.sql("""
    SELECT *  FROM patent WHERE filename IS NULL or filename=''
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+----+------+-------+----+--------+-----+----+----------+--------+---------+
| id|type|number|country|date|abstract|title|kind|num_claims|filename|withdrawn|
+---+----+------+-------+----+--------+-----+----+----------+--------+---------+
+---+----+------+-------+----+--------+-----+----+----------+--------+---------+

**Observations**:
- All patents have filename

### Analyse field withdrawn

In [21]:
spark.sql("""
    SELECT withdrawn, count(*)  FROM patent GROUP BY withdrawn
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------+
|withdrawn|count(1)|
+---------+--------+
|        1|   13600|
|        0| 7977458|
+---------+--------+

**Observations**:
- 0.17% of the patents are withdrawn

## Exploratory Analysis on Patent Assignee

In [22]:
patent_assignee_path = "s3://patents-analytics/raw/patent_assignee.tsv"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
patent_assignee = spark.read.options(header='True', delimiter='\t', inferSchema='True', escape="\"", multiline='True').csv(patent_assignee_path).repartition(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
patent_assignee.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(patent_id='9098154', assignee_id='bec1d9ba-7ad6-48cc-9191-3c644c0d9a56', location_id='ed8be594-09bd-11ec-893a-12de62d610b1'), Row(patent_id='6241143', assignee_id='6f8a613b-f7a6-4991-ba1a-f57e00ccc607', location_id='ff405b60-cb8e-11eb-9615-121df0c29c1e'), Row(patent_id='8423973', assignee_id='2a04f703-2243-4deb-a0fa-8ecec900f529', location_id='3fac045d-cb8e-11eb-9615-121df0c29c1e'), Row(patent_id='9731019', assignee_id='8cf738cf-7134-427e-a550-1c217cebeceb', location_id='f1b6c93e-cb8e-11eb-9615-121df0c29c1e'), Row(patent_id='5927243', assignee_id='f4b333a6-bc14-460e-b4ca-6d9454cf1db2', location_id='fef7fa78-cb90-11eb-9615-121df0c29c1e')]

In [25]:
patent_assignee.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- patent_id: string (nullable = true)
 |-- assignee_id: string (nullable = true)
 |-- location_id: string (nullable = true)

In [26]:
patent_assignee.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7334687

In [27]:
patent_assignee.createOrReplaceTempView('patent_assignee')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Analyse field patent_id

In [31]:
spark.sql("""
    SELECT COUNT(DISTINCT(patent_id)) as count_patent_id FROM patent_assignee
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+
|count_patent_id|
+---------------+
|        7084751|
+---------------+

**Observations**:
- The number of unique patent id is only 7084751, which means there are 906,307 patents with no assignee

In [34]:
spark.sql("""
    SELECT * FROM patent LEFT JOIN patent_assignee ON patent.id = patent_assignee.patent_id WHERE patent_assignee.patent_id is NULL
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+--------+-------+-------------------+--------------------+--------------------+----+----------+-------------+---------+---------+-----------+-----------+
|      id|   type|  number|country|               date|            abstract|               title|kind|num_claims|     filename|withdrawn|patent_id|assignee_id|location_id|
+--------+-------+--------+-------+-------------------+--------------------+--------------------+----+----------+-------------+---------+---------+-----------+-----------+
|10000304|utility|10000304|     US|2018-06-19 00:00:00|The present discl...|Method for saniti...|  B2|        14|ipg180619.xml|        0|     null|       null|       null|
|10002552|utility|10002552|     US|2018-06-19 00:00:00|A light panel dis...|Light panel displ...|  B1|        17|ipg180619.xml|        0|     null|       null|       null|
|10002595|utility|10002595|     US|2018-06-19 00:00:00|A wind instrument...|Rhino resonator a...|  B2|        16|ipg180619.xml|        0|   

In [35]:
spark.sql("""
    SELECT * FROM patent RIGHT JOIN patent_assignee ON patent.id = patent_assignee.patent_id WHERE patent.id is NULL
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+----+------+-------+----+--------+-----+----+----------+--------+---------+---------+-----------+-----------+
| id|type|number|country|date|abstract|title|kind|num_claims|filename|withdrawn|patent_id|assignee_id|location_id|
+---+----+------+-------+----+--------+-----+----+----------+--------+---------+---------+-----------+-----------+
+---+----+------+-------+----+--------+-----+----+----------+--------+---------+---------+-----------+-----------+

**Observations**:
- All patent id in the patent_assignee table are found in patent table

### Analyse field assignee id

In [36]:
spark.sql("""
    SELECT * FROM patent_assignee WHERE assignee_id IS NULL or assignee_id =''
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-----------+-----------+
|patent_id|assignee_id|location_id|
+---------+-----------+-----------+
+---------+-----------+-----------+

### Analyse field location id

In [37]:
spark.sql("""
    SELECT * FROM patent_assignee WHERE location_id IS NULL or location_id =''
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+-----------+
|patent_id|         assignee_id|location_id|
+---------+--------------------+-----------+
| 11121230|8c265a09-164a-4c1...|       null|
|  9273411|df0a8e74-6099-450...|       null|
|  7175684|3569fbc5-25c8-4b7...|       null|
|  9394589|d71eabf4-1c32-4bb...|       null|
|  8116931|85d586ac-cbda-44b...|       null|
|  7852445|41ff8c06-dc7b-4f6...|       null|
|  8032989|ffbd2f08-b39f-4b0...|       null|
|  D644083|f80edb85-5f36-432...|       null|
|  8919099|85d586ac-cbda-44b...|       null|
|  7630440|4eab510f-771b-4d8...|       null|
| 11088486|c459201c-5993-405...|       null|
|  8593403|a99ebb70-0ab9-4bc...|       null|
|  RE41585|33757ce1-f2df-4bc...|       null|
|  D924109|31580c0f-8eab-49e...|       null|
|  6899863|0aeaa72e-18ca-482...|       null|
|  7825052|8ff48169-9291-4e4...|       null|
|  8175781|85d586ac-cbda-44b...|       null|
|  8259845|151087a6-ae76-469...|       null|
|  8539933|85d586ac-cbda-44b...|       null|
|  9712677

**Observations**:
- Some locations are missing

### Check whether there is any duplicate

In [39]:
spark.sql("""SELECT patent_id, assignee_id, location_id,  COUNT(*)
    FROM patent_assignee
    GROUP BY patent_id, assignee_id, location_id
    HAVING COUNT(*) > 1
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+--------------------+--------+
|patent_id|         assignee_id|         location_id|count(1)|
+---------+--------------------+--------------------+--------+
|  8913287|4a7adfa5-124c-4f3...|ff4052ad-cb90-11e...|       2|
|  8131177|4a7adfa5-124c-4f3...|ff4052ad-cb90-11e...|       2|
| 10064485|63ed96b5-cb45-45a...|ff010a8c-cb8f-11e...|       2|
|  4691509|f87620f1-eeaf-46f...|fdacbf44-cb8f-11e...|       2|
|  9827781|4a7adfa5-124c-4f3...|ff4052ad-cb90-11e...|       2|
|  6858639|6021d0f9-f8e8-42b...|ff4052ad-cb90-11e...|       2|
|  7737243|1d1a8301-3e0c-416...|                null|       2|
|  D553748|4a7adfa5-124c-4f3...|fc671a1c-cb8e-11e...|       2|
|  6977736|4a7adfa5-124c-4f3...|ff4052ad-cb90-11e...|       2|
|  4955327|6c890ba7-0444-42f...|ff4052ad-cb90-11e...|       2|
|  8508822|4a7adfa5-124c-4f3...|ff4052ad-cb90-11e...|       2|
|  8228573|4a7adfa5-124c-4f3...|ff4052ad-cb90-11e...|       2|
|  7577381|4a7adfa5-124c-4f3...|ff4052ad-cb90-11e...|  

**Observations**:
- There are some duplicates in this patent_assignee

## Exploratory Analysis on Assignee

In [40]:
assignee_path = "s3://patents-analytics/raw/assignee.tsv"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
assignee = spark.read.options(header='True', delimiter='\t', inferSchema='True', escape="\"", multiline='True').csv(assignee_path).repartition(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
assignee.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(id='cf06cbad-b4f3-45f7-888a-c0d078f5b856', type=2.0, name_first=None, name_last=None, organization='Rainbow Display, Inc.'), Row(id='4f18552a-5db7-4b04-a17c-14621545f46d', type=2.0, name_first=None, name_last=None, organization='LECO Corporation'), Row(id='24effb01-7bc0-43c6-beea-9b96fec05d75', type=2.0, name_first=None, name_last=None, organization='Sper Scientific Ltd'), Row(id='e47b6be9-98b3-46c6-8dcd-9b15bd0a6614', type=2.0, name_first=None, name_last=None, organization='Amenex Associates, Inc.'), Row(id='91d19128-6b46-4546-9c35-8b0679dfae66', type=3.0, name_first=None, name_last=None, organization='Living Technologies, Cooperatieve Vennootschap Met Beperkteaansprakelijkheid')]

In [43]:
assignee.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- type: double (nullable = true)
 |-- name_first: string (nullable = true)
 |-- name_last: string (nullable = true)
 |-- organization: string (nullable = true)

In [45]:
assignee.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

540183

In [44]:
assignee.createOrReplaceTempView('assignee')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Check duplicates

In [46]:
spark.sql("""
    SELECT COUNT(DISTINCT(ID)) FROM assignee
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+
|count(DISTINCT ID)|
+------------------+
|            540183|
+------------------+

**Observations**:
    - No duplicates

### Analyse field type

In [48]:
spark.sql("""
    SELECT type, count(*) FROM assignee GROUP BY type
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------+
|type|count(1)|
+----+--------+
|null|   14220|
| 1.0|       2|
|13.0|     143|
|17.0|       2|
| 6.0|     250|
|15.0|     337|
| 5.0|   21538|
| 2.0|  219182|
| 4.0|   29931|
|14.0|    1809|
| 8.0|      12|
| 0.0|      78|
| 7.0|     699|
| 3.0|  251764|
| 9.0|      21|
|12.0|     195|
+----+--------+

**Observations**:
- There are 14,220 assignee with no type
- Based on the data dictionary, type ranges from 1 to 9. If there is "1" appearing before the codes signifies part interest.


### Analyse name_first and name_last and organisation

In [51]:
spark.sql("""
    SELECT * FROM assignee WHERE (type=4 or type=5) and ((name_first IS NULL or name_first='') OR (name_last IS NULL or name_last=''))
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+--------------------+---------+--------------------+
|                  id|type|          name_first|name_last|        organization|
+--------------------+----+--------------------+---------+--------------------+
|463a3ff8-fe60-4fb...| 4.0|       NOVOZYMES A/S|     null|                null|
|2521098e-ecd9-46b...| 4.0|Shenzhen Raysees ...|     null|                null|
|42a22cb3-1e13-4bf...| 4.0|                null|     null|Victoria and Olee...|
|9dce7efd-904f-469...| 4.0|GEBERIT INTERNATI...|     null|                null|
|f83254bb-5101-4df...| 4.0|                null|     null|  Helmut E. W. Masch|
|895c19c7-6181-46a...| 4.0|                null|     null| William J. McNamara|
|b85d6049-d511-4f1...| 4.0|                null|     null|      Mervyn Shapiro|
|50578afa-ccb2-416...| 4.0|Wonderland Switze...|     null|                null|
|baeacd89-4276-42a...| 4.0|TAIWAN SEMICONDUC...|     null|                null|
|5b855ed1-81e0-46a...| 4.0|             

**Observations**:
- Some individual assignee name is placed in the organization instead of name_first and name_last

In [52]:
spark.sql("""
    SELECT * FROM assignee WHERE type!=4 and type!=5 and name_first IS NOT NULL AND name_last IS NOT NULL
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+--------------------+-----------+------------+
|                  id|type|          name_first|  name_last|organization|
+--------------------+----+--------------------+-----------+------------+
|d010f495-dff2-4f4...|15.0|       Howard Ingram|     Cairns|        null|
|913a43da-9d99-4d7...| 2.0|                 Jed|     Gaines|        null|
|af97c6d8-d161-497...|14.0|           Samuel R.|     Carter|        null|
|44be01e8-3fe9-47c...| 2.0|       Miss. Gregory|    Valente|        null|
|e49b3161-3d1f-414...| 2.0|       Mr. Alexander|    Fridman|        null|
|3a8e7332-f9b4-4e8...| 3.0|Timfold-es Mukoru...|Magyarovari|        null|
|299c10b5-c17e-4e0...| 2.0|               Henry|       Lobe|        null|
|d2f6adb4-cb81-41c...|14.0|          Charles W.|     Pierce|        null|
|53323de1-fc87-4d7...|14.0|              Joseph|     Regard|        null|
|51ea1c5d-65aa-46a...|15.0|                Gina|        Lie|        null|
|9d0b75ed-66c3-48a...|14.0|           

In [53]:
spark.sql("""
    SELECT * FROM assignee WHERE organization is not null and name_first is not null and name_last is not null
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+------------+------------+--------------------+
|                  id|type|  name_first|   name_last|        organization|
+--------------------+----+------------+------------+--------------------+
|5e94b55d-681a-4d2...| 6.0|     William|Lockett, III|Tennesee Valley A...|
|c6cd5696-6b8c-4eb...| 6.0|     William|Lockett, III|The United States...|
|0fc8a6e5-04d6-4ae...| 2.0|    James P.|    Liautaud|Bell Telephone La...|
|ff8e39c4-7f4f-480...| 2.0|     William|Lockett, III|AT&T Bell Laborat...|
|69d57865-cf4f-454...| 6.0|     William|Lockett, III|The United States...|
|4f299bfd-8d2a-487...| 2.0|    James P.|    Liautaud|AT&T Technologies...|
|6824d7c9-8f1f-4bd...| 2.0|Frederick H.|       Kroll|   W. R. Grace & Co.|
|3103651b-f835-411...| 7.0|    James P.|    Liautaud|The United States...|
|29a08113-069e-49f...| 6.0|     William|Lockett, III|The United States...|
|38cc49c5-f655-4a9...| 3.0|    James P.|    Liautaud|BBC Brown, Boveri...|
|479eb83f-4741-4d8...| 2.

**Observations**:
    - Some patent have individual name as well as the organisation name

In [55]:
spark.sql("""
    SELECT * FROM assignee WHERE organization is null and name_first is null and name_last is null
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+----+----------+---------+------------+
| id|type|name_first|name_last|organization|
+---+----+----------+---------+------------+
+---+----+----------+---------+------------+

**Observations**:
- All the rows has either name or organisation

## Exploratory Analysis on Location

In [57]:
location_path = "s3://patents-analytics/raw/location.tsv"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
location = spark.read.options(header='True', delimiter='\t', inferSchema='True', escape="\"", multiline='True').csv(location_path).repartition(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
location.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(id='e10f3d8a-cb8e-11eb-9615-121df0c29c1e', city='Saint Laurant du Var', state=None, country='FR', latitude=46.2621, longitude=-0.21006, county=None, state_fips=None, county_fips=None), Row(id='02b3429f-cb90-11eb-9615-121df0c29c1e', city='Zornhcim', state=None, country='DE', latitude=None, longitude=None, county=None, state_fips=None, county_fips=None), Row(id='bfe59c31-cb8e-11eb-9615-121df0c29c1e', city='Doddanankundi', state=None, country='IN', latitude=None, longitude=None, county=None, state_fips=None, county_fips=None), Row(id='8147f6e3-cb8e-11eb-9615-121df0c29c1e', city='Stella-Plage', state=None, country='FR', latitude=50.4799, longitude=1.57714, county=None, state_fips=None, county_fips=None), Row(id='fa670c89-cb8e-11eb-9615-121df0c29c1e', city='Naremburn', state=None, country='AU', latitude=-33.8171, longitude=151.202, county=None, state_fips=None, county_fips=None)]

In [60]:
location.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

242926

In [61]:
location.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- county: string (nullable = true)
 |-- state_fips: double (nullable = true)
 |-- county_fips: integer (nullable = true)

In [62]:
location.createOrReplaceTempView('location')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Analyse field city

In [63]:
spark.sql("""
    SELECT * from location WHERE city IS NULL or city=''
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+-----+-------+--------+---------+------+----------+-----------+
|                  id|city|state|country|latitude|longitude|county|state_fips|county_fips|
+--------------------+----+-----+-------+--------+---------+------+----------+-----------+
|f965f0dd-cb90-11e...|null|   AK|     US| 61.9328| -153.723|  null|       2.0|       null|
|f192fea1-cb8e-11e...|null| null|     NE| 17.6049|  9.90205|  null|      null|       null|
|535779c9-cb8e-11e...|null|   NV|     US| 38.5031| -116.667|  null|      32.0|       null|
|fbfbf758-cb8f-11e...|null|   AZ|     US| 34.1685| -111.669|  null|       4.0|       null|
|aa58c5bc-cb90-11e...|null| null|     SC|-4.29879|  55.5891|  null|      null|       null|
|7c99c93f-cb90-11e...|null| null|     KZ| 48.0069|  66.3074|  null|      null|       null|
|28fe4971-cb8f-11e...|null| null|     NG| 8.97718|  7.80771|  null|      null|       null|
|0738d5fb-cb91-11e...|null| null|     GC|    null|     null|  null|      null|       null|

**Observations**:
- There are some missing values in the city, state, latitude, longitude, country, state_fips, county_fips

### Analyse field Country

In [64]:
spark.sql("""
    SELECT * from location WHERE country IS NULL or country=''
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+-------+--------+---------+------+----------+-----------+
|                  id|                city|state|country|latitude|longitude|county|state_fips|county_fips|
+--------------------+--------------------+-----+-------+--------+---------+------+----------+-----------+
|b9dd4981-09bc-11e...|                Hilo|   HI|   null| 19.6854|   -155.1|  null|      null|       null|
|327f81b8-09bd-11e...|            Longmont| null|   null| 49.2933|  2.74558|  null|      null|       null|
|4a1aa0df-cb8e-11e...|          Birmingham|   AL|   null|    null|     null|  null|      null|       null|
|8c7eed53-cb8e-11e...|         Charmichael|   CA|   null|    null|     null|  null|      null|       null|
|980f301c-cb8f-11e...|Haneda, Ota-ku, T...| null|   null| 35.5498|  139.786|  null|      null|       null|
|0730c945-09bd-11e...|     Kibbutz Yizrael| null|   null|    null|     null|  null|      null|       null|
|82329f91-cb8e-11e...|          Louis

**Observations**:
    - Some data has missing country, but have city or state, or latitude and longitude provided, we may be able to derive the country value from this information

In [65]:
spark.sql("""
    SELECT * from location WHERE city is null and state is null and country is null and latitude is null and longitude is null and county is null and state_fips is null and county_fips is null
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+-----+-------+--------+---------+------+----------+-----------+
|                  id|city|state|country|latitude|longitude|county|state_fips|county_fips|
+--------------------+----+-----+-------+--------+---------+------+----------+-----------+
|3cb80671-cb8e-11e...|null| null|   null|    null|     null|  null|      null|       null|
|73f1845d-cb8e-11e...|null| null|   null|    null|     null|  null|      null|       null|
+--------------------+----+-----+-------+--------+---------+------+----------+-----------+

**Observations**:
- There are two data which does not have any location information at all

## Exploratory Analysis on WIPO

In [66]:
wipo_path = "s3://patents-analytics/raw/wipo.tsv"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
wipo = spark.read.options(header='True', delimiter='\t', inferSchema='True', escape="\"", multiline='True').csv(wipo_path).repartition(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
wipo.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(patent_id=9219777, field_id=6.0, sequence=1), Row(patent_id=6262715, field_id=6.0, sequence=0), Row(patent_id=10123471, field_id=29.0, sequence=0), Row(patent_id=8779043, field_id=17.0, sequence=0), Row(patent_id=10281292, field_id=12.0, sequence=0)]

In [69]:
wipo.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10658264

In [70]:
wipo.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- patent_id: integer (nullable = true)
 |-- field_id: double (nullable = true)
 |-- sequence: integer (nullable = true)

In [71]:
wipo.createOrReplaceTempView('wipo')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Check duplicates

In [73]:
spark.sql("""
    SELECT patent_id, field_id, sequence, count(*) as count from wipo GROUP BY patent_id, field_id, sequence HAVING count>1
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------+--------+-----+
|patent_id|field_id|sequence|count|
+---------+--------+--------+-----+
+---------+--------+--------+-----+

**Observations**:
- no duplicates

### Analyse field patent_id

In [72]:
spark.sql("""
    SELECT COUNT(DISTINCT(patent_id)) from wipo
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------+
|count(DISTINCT patent_id)|
+-------------------------+
|                  7221215|
+-------------------------+

In [76]:
spark.sql("""
    SELECT * from patent LEFT JOIN wipo ON patent.id = wipo.patent_id WHERE wipo.patent_id IS NULL
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------+-------+-------+-------------------+--------------------+--------------------+----+----------+--------------------+---------+---------+--------+--------+
|     id|   type| number|country|               date|            abstract|               title|kind|num_claims|            filename|withdrawn|patent_id|field_id|sequence|
+-------+-------+-------+-------+-------------------+--------------------+--------------------+----+----------+--------------------+---------+---------+--------+--------+
|3962445|utility|3962445|     US|1976-06-08 00:00:00| A class of 4-hyd...|Anti-allergenic c...|   A|        58|pftaps19760608_wk...|        0|     null|    null|    null|
|3981900|utility|3981900|     US|1976-09-21 00:00:00| The addition of ...|Preparation of al...|   A|         9|pftaps19760921_wk...|        0|     null|    null|    null|
|3998879|utility|3998879|     US|1976-12-21 00:00:00| A new class of a...|Arylsulfonylcarba...|   A|         4|pftaps19761221_wk...|        0|   

**Observations**:
- There are some patent that do not have WIPO classification

In [77]:
spark.sql("""
    SELECT * from patent RIGHT JOIN wipo ON patent.id = wipo.patent_id WHERE patent.id IS NULL
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+----+------+-------+----+--------+-----+----+----------+--------+---------+---------+--------+--------+
| id|type|number|country|date|abstract|title|kind|num_claims|filename|withdrawn|patent_id|field_id|sequence|
+---+----+------+-------+----+--------+-----+----+----------+--------+---------+---------+--------+--------+
+---+----+------+-------+----+--------+-----+----+----------+--------+---------+---------+--------+--------+

**Observations**:
- All patent_id in WIPO table can be found in patent table

### Analyse field_id

In [90]:
spark.sql("""
    SELECT distinct(field_id) from wipo
""").show(50)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|field_id|
+--------+
|    29.0|
|     1.0|
|    28.0|
|    13.0|
|    26.0|
|    21.0|
|    17.0|
|    27.0|
|     6.0|
|    20.0|
|    15.0|
|     5.0|
|    35.0|
|    25.0|
|    23.0|
|    19.0|
|     2.0|
|    24.0|
|    34.0|
|     4.0|
|    31.0|
|    14.0|
|    10.0|
|    30.0|
|    32.0|
|     8.0|
|     7.0|
|    18.0|
|    11.0|
|    22.0|
|     3.0|
|    33.0|
|     9.0|
|    16.0|
|    12.0|
+--------+

**Observations**:
- The field_id ranges from 1 to 35 (in line with WIPO classification)

### Analyse sequence

In [82]:
spark.sql("""
    SELECT DISTINCT(sequence) from wipo
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|sequence|
+--------+
|       1|
|       6|
|       3|
|       4|
|       5|
|       2|
|       7|
|       0|
+--------+

**Observations**:
- The classification sequence ranges from 0 to 7

## Exploratory Analysis on WIPO_FIELD

In [83]:
wipo_field_path = "s3://patents-analytics/raw/wipo_field.tsv"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:
wipo_field = spark.read.options(header='True', delimiter='\t', inferSchema='True', escape="\"", multiline='True').csv(wipo_field_path).repartition(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [86]:
wipo_field.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(id='6', sector_title='Electrical engineering', field_title='Computer technology'), Row(id='D15', sector_title='Design', field_title='Machines, not elsewhere specified'), Row(id='9', sector_title='Instruments', field_title='Optics'), Row(id='D34', sector_title='Design', field_title='Material or Article Handling Equipment'), Row(id='34', sector_title='Other fields', field_title='Other consumer goods')]

In [87]:
wipo_field.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- sector_title: string (nullable = true)
 |-- field_title: string (nullable = true)

In [89]:
wipo_field.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

70

In [93]:
wipo_field.createOrReplaceTempView('wipo_field')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Analyse id

In [94]:
spark.sql("""
    SELECT DISTINCT(id) from wipo_field
""").show(70)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+
| id|
+---+
|  7|
|D27|
|D20|
|D03|
| 31|
| 18|
| 27|
| 26|
|D31|
| 25|
| 33|
|D08|
|  9|
|D10|
| 36|
|D13|
|D28|
|D23|
| 15|
|D29|
|D34|
| 29|
|D32|
| 23|
|D26|
|D01|
| 13|
| 14|
|D16|
|D11|
|  3|
| 30|
| 34|
|D05|
| 28|
| 22|
|D12|
|D15|
|D06|
|D30|
| 24|
|D14|
|D21|
|D18|
| 16|
| 35|
| 17|
|  6|
|D24|
|D25|
|D19|
|D17|
|D02|
|  1|
| 20|
| 21|
|D22|
| 11|
|D09|
|  8|
|  5|
|D04|
|D07|
| 19|
| 32|
| 10|
| 12|
|  4|
|  2|
|D99|
+---+

**Observations**:
- In the wipo table the field_id only ranges from 1-35, there is no field_id which starts with D

### Analyse sector_title

In [95]:
spark.sql("""
    SELECT sector_title, count(*) from wipo_field GROUP BY sector_title
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------+
|        sector_title|count(1)|
+--------------------+--------+
|Mechanical engine...|       8|
|        Other fields|       3|
|              Design|      34|
|         Instruments|       5|
|           Chemistry|      11|
|Electrical engine...|       8|
|               Plant|       1|
+--------------------+--------+

In [98]:
spark.sql("""
    SELECT * from wipo_field WHERE sector_title='Design'
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------------+--------------------+
| id|sector_title|         field_title|
+---+------------+--------------------+
|D15|      Design|Machines, not els...|
|D18|      Design|Printing and offi...|
|D06|      Design|          Furnishing|
|D04|      Design|           Brushware|
|D14|      Design|Recording, commun...|
|D11|      Design|Articles of adorn...|
|D19|      Design|Stationery and of...|
|D21|      Design|Games, toys, tent...|
|D07|      Design|Household goods, ...|
|D26|      Design|  Lighting apparatus|
|D13|      Design|Equipment for pro...|
|D17|      Design| Musical instruments|
|D22|      Design|Arms, pyrotechnic...|
|D10|      Design|Clocks and watche...|
|D31|      Design|Machines and appl...|
|D29|      Design|Devices and equip...|
|D01|      Design|          Foodstuffs|
|D23|      Design|Fluid distributio...|
|D34|      Design|Material or Artic...|
|D05|      Design|Textile piecegood...|
+---+------------+--------------------+
only showing top 20 rows

### Analyse field_title

In [97]:
spark.sql("""
    SELECT * from wipo_field where field_title IS NULL or field_title=''
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------------+-----------+
| id|sector_title|field_title|
+---+------------+-----------+
+---+------------+-----------+

**Observations**:
- There is no missing value in field_title